In [1]:
import scanpy as sc
import pandas as pd
import os
from dca.api import dca
import tensorflow as tf
from tensorflow.python.keras.backend import set_session

2021-08-02 11:19:24.815540: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-02 11:19:24.815557: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/users/mscherer/software/anaconda3/envs/dca/lib/python3.8/site-packages/kopt/config.py:60: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  _config = yaml.load(open(_config_path))


### INPUT: Specify which sample you want to analyze. Currently available are:

- Sample2_70_percent
- Sample2_80_percent
- Sample3_default
- Sample4_70_percent
- Sample4_80_percent
- Sample5_70_percent
- Sample5_80_percent

In [2]:
sample = 'Sample5_80_percent'

### Loading the read count data and potentially removing cell doublets/mulitplets.

In [3]:
dat = pd.read_table('/users/lvelten/project/Methylome/analysis/missionbio/tapestri/' + sample + '/tsv/' + sample + '.barcode.cell.distribution.tsv',sep='\t')
remove_mixed = True
if remove_mixed:
    clust_file = pd.read_csv('/users/lvelten/project/Methylome/analysis/missionbio/tapestri/'+ sample + '/tsv/cluster_assignment.tsv',
                            sep='\t')
    drop_rows = clust_file['Barcode'][clust_file['CellType']=='Mixed']
    dat = dat.drop(drop_rows)

### Creating an AnnData object from the read counts.

In [4]:
adata = sc.AnnData(dat)
sc.pp.filter_genes(adata,min_cells=1)

### Running the autoencoder: Currently, we have the following available architectures:

- 'meth-encoder': A classical autoencoder, where the dispersion parameters of both NB-distributions can be freely selected by the model.
- 'meth-encoder-constant': An autoencoder, where the dispersion parameters is fixed for each gene. This lowers the number of paramters substantially.
- 'meth-encoder-poisson': An autoencoder, which mixes a negative bionomial distribution (foreground) with a Poisson distribution as the background
- 'meth-encoder-poisson-constant': Same as the above, but with a fixed dispersion for each gene
- 'meth-encoder-simple': Instead of having a dense layer representing the mean per cell and amplicon, we use a constant mean per amplicon for both the foreground and the background distribution.

In [5]:
res = dca(adata,
          ae_type='meth-encoder-constant',
          return_info=True,
          return_model=True,
          return_bottleneck=True,
          verbose=True,
          early_stop=50,
          network_kwds={'debug': True},
          init='glorot_normal',
          hidden_size=(16,8,16))

2021-08-02 11:19:26.762338: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-02 11:19:26.762485: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-02 11:19:26.762498: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-02 11:19:26.762514: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CZC041BTPK): /proc/driver/nvidia/version does not exist


dca: Successfully preprocessed 199 genes and 5207 cells.



2021-08-02 11:19:26,948 [WARNING] From /users/lvelten/project/Methylome/src/error_mod/dca/dca/train.py:41: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

2021-08-02 11:19:26.949711: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
count (InputLayer)              [(None, 199)]        0                                            
__________________________________________________________________________________________________
enc0 (Dense)                    (None, 16)           3200        count[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16)           48          enc0[0][0]                       
__________________________________________________________________________________________________
enc0_act (Activation)           (None, 16)           0           batch_normalization[0][0]        
____________________________________________________________________________________________

2021-08-02 11:19:27.840429: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2021-08-02 11:19:27.859988: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz


4686/4686 [==============================] - 1s 149us/sample - loss: 465.7851 - val_loss: 294.7353
Epoch 2/300
1152/4686 [======>.......................] - ETA: 0s - loss: 294.7743

/users/mscherer/software/anaconda3/envs/dca/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


4686/4686 [==============================] - 0s 44us/sample - loss: 221.6957 - val_loss: 162.3700
Epoch 3/300
4686/4686 [==============================] - 0s 43us/sample - loss: 87.6008 - val_loss: 63.2342
Epoch 4/300
4686/4686 [==============================] - 0s 42us/sample - loss: 26.7032 - val_loss: 14.7649
Epoch 5/300
4686/4686 [==============================] - 0s 43us/sample - loss: 9.7909 - val_loss: 7.8985
Epoch 6/300
4686/4686 [==============================] - 0s 45us/sample - loss: 7.4881 - val_loss: 7.2293
Epoch 7/300
4686/4686 [==============================] - 0s 43us/sample - loss: 7.0559 - val_loss: 6.9814
Epoch 8/300
4686/4686 [==============================] - 0s 42us/sample - loss: 6.8117 - val_loss: 6.7545
Epoch 9/300
4686/4686 [==============================] - 0s 42us/sample - loss: 6.6561 - val_loss: 6.6137
Epoch 10/300
4686/4686 [==============================] - 0s 43us/sample - loss: 6.5399 - val_loss: 6.5152
Epoch 11/300
4686/4686 [=========================

Epoch 76/300
4686/4686 [==============================] - 0s 43us/sample - loss: 6.2294 - val_loss: 6.2668
Epoch 77/300
4686/4686 [==============================] - 0s 42us/sample - loss: 6.2285 - val_loss: 6.2669
Epoch 78/300
4686/4686 [==============================] - 0s 42us/sample - loss: 6.2288 - val_loss: 6.2668
Epoch 79/300
4686/4686 [==============================] - 0s 42us/sample - loss: 6.2291 - val_loss: 6.2671
Epoch 80/300
4686/4686 [==============================] - 0s 44us/sample - loss: 6.2277 - val_loss: 6.2667
Epoch 81/300
4686/4686 [==============================] - 0s 52us/sample - loss: 6.2288 - val_loss: 6.2666
Epoch 82/300
4686/4686 [==============================] - 0s 50us/sample - loss: 6.2283 - val_loss: 6.2667
Epoch 83/300
3712/4686 [======================>.......] - ETA: 0s - loss: 6.2328
Epoch 00083: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
4686/4686 [==============================] - 0s 44us/sample - loss: 6.2283 - val_loss: 6.

### Writing the data on disk.

In [6]:
di = '/users/lvelten/project/Methylome/analysis/missionbio/tapestri/' + sample + '/methylation_autoencoder/'
if not os.path.isdir(di):
    os.mkdir(di)
    
pd.DataFrame(adata.obsm['X_meth_value']).to_csv(di + 'mixture_prob_dca.csv')

In [7]:
foo = res.get_encoder().predict([adata.X, adata.X.mean(1)])

In [8]:
pd.DataFrame(adata.obsm['X_bottleneck']).to_csv(di + 'bottleneck.csv')